In [1]:
import os
import os.path as osp
import sys
import time

current_directory = os.getcwd()
sys.path.append(current_directory)

import mmcv
import torch
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.runner import (load_checkpoint,
                         wrap_fp16_model)

from mmseg.apis import single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import build_dp, get_device, setup_multi_processes

/root/miniconda3/envs/SCTNet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/SCTNet/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
# config_path = r"configs/sctnet/ADE20K/sctnet-b_8x4_160k_ade.py"
# checkpoint = r"configs/sctnet/ADE20K/pretrain/SCTNet-B-ADE20K.pth"

# config_path = r"configs/sctnet/COCO-Stuff-10K/sctnet_b_4x4_160k.py"
# checkpoint = r"configs/sctnet/COCO-Stuff-10K/pretrain/SCTNet-B_COCO-Stuff-10K.pth"

config_path = r"configs/sctnet/pets/sctnet-b_8x4_160k_pets.py"
checkpoint = r"work_dirs/sctnet-b_8x4_160k_pets/latest.pth"

cfg = mmcv.Config.fromfile(config_path)

# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

cfg.gpu_ids = [0]

In [3]:
# mkdir work_dir
work_dir = osp.join('./work_dirs', osp.splitext(osp.basename(config_path))[0])
mmcv.mkdir_or_exist(osp.abspath(work_dir))
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
json_file = osp.join(work_dir, f'eval_single_scale_{timestamp}.json')
output_img_dir = osp.join(work_dir, f'eval_single_scale_{timestamp}')

In [4]:
# build the dataloader
# TODO: support multiple images per gpu (only minor changes are needed)
dataset = build_dataset(cfg.data.test)
# The default loader config
loader_cfg = dict(
    # cfg.gpus will be ignored if distributed
    num_gpus=len(cfg.gpu_ids),
    dist=False,
    shuffle=False)
# The overall dataloader settings
loader_cfg.update({
    k: v
    for k, v in cfg.data.items() if k not in [
        'train', 'val', 'test', 'train_dataloader', 'val_dataloader',
        'test_dataloader'
    ]
})
test_loader_cfg = {
    **loader_cfg,
    'samples_per_gpu': 1,
    'shuffle': False,  # Not shuffle by default
    **cfg.data.get('test_dataloader', {})
}
# build the dataloader
data_loader = build_dataloader(dataset, **test_loader_cfg)

2024-11-29 08:07:45,363 - mmseg - INFO - Loaded 740 images


In [5]:
# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
if 'CLASSES' in checkpoint.get('meta', {}):
    model.CLASSES = checkpoint['meta']['CLASSES']
else:
    print('"CLASSES" not found in meta, use dataset.CLASSES instead')
    model.CLASSES = dataset.CLASSES
if 'PALETTE' in checkpoint.get('meta', {}):
    model.PALETTE = checkpoint['meta']['PALETTE']
else:
    print('"PALETTE" not found in meta, use dataset.PALETTE instead')
    model.PALETTE = dataset.PALETTE

# clean gpu memory when starting a new evaluation.
torch.cuda.empty_cache()

cfg.device = get_device()

model = revert_sync_batchnorm(model)
model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)

2024-11-29 08:07:48,435 - mmseg - INFO - initialize SCTNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'pretrain/SCT-B_Pretrain.pth'}
2024-11-29 08:07:48,437 - mmcv - INFO - load model from: pretrain/SCT-B_Pretrain.pth
2024-11-29 08:07:48,439 - mmcv - INFO - load checkpoint from local path: pretrain/SCT-B_Pretrain.pth
/root/workspace/SCTNet/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
2024-11-29 08:07:49,552 - mmseg - INFO - initialize VitGuidanceHead with init_cfg {'type': 'Pretrained', 'checkpoint': 'pretrain/Teacher_SegFormer_B3_ADE.pth'}
2024-11-29 08:07:49,553 - mmcv - INFO - load model from: pretrain/Teacher_SegFormer_B3_ADE.pth
2024-11-29 08:07:49,553 - mmcv - INFO - load checkpoint from local path: pretrain/Teacher_SegFormer_B3_ADE.pth

load checkpoint from local path: work_dirs/sctnet-b_8x4_160k_pets/iter_108000.pth


In [6]:
from mmcv.image import tensor2imgs

out_dir = './work_dirs'

eval_kwargs = {}

model.eval()

results = []
dataset = data_loader.dataset
prog_bar = mmcv.ProgressBar(len(dataset))
# The pipeline about how the data_loader retrieval samples from dataset:
# sampler -> batch_sampler -> indices
# The indices are passed to dataset_fetcher to get data from dataset.
# data_fetcher -> collate_fn(dataset[index]) -> data_sample
# we use batch_sampler to get correct data idx
loader_indices = data_loader.batch_sampler

for batch_indices, data in zip(loader_indices, data_loader):
    with torch.no_grad():
        result = model(return_loss=False, **data)

    # draw results
    img_tensor = data['img'][0]
    img_metas = data['img_metas'][0].data[0]
    imgs = tensor2imgs(img_tensor, **img_metas[0]['img_norm_cfg'])
    assert len(imgs) == len(img_metas)

    if out_dir:
        for img, img_meta in zip(imgs, img_metas):
            h, w, _ = img_meta['img_shape']
            img_show = img[:h, :w, :]

            ori_h, ori_w = img_meta['ori_shape'][:-1]
            img_show = mmcv.imresize(img_show, (ori_w, ori_h))

            if out_dir:
                out_file = osp.join(out_dir, img_meta['ori_filename'])
            else:
                out_file = None

            model.module.show_result(
                img_show,
                result,
                palette=dataset.PALETTE,
                show=True,
                out_file=out_file,
                opacity=0.5)

    results.extend(result)
    batch_size = len(result)
    for _ in range(batch_size):
        prog_bar.update()

eval_kwargs.update(metric="mIoU")
metric = dataset.evaluate(results, **eval_kwargs)
metric_dict = dict(config=config_path, metric=metric)
mmcv.dump(metric_dict, json_file, indent=4)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 740/740, 15.3 task/s, elapsed: 48s, ETA:     0sper class results:

+-------+-------+-------+
| Class |  IoU  |  Acc  |
+-------+-------+-------+
|  cat  | 83.02 | 89.72 |
|  dog  | 90.59 | 94.15 |
+-------+-------+-------+
Summary:

+-------+------+-------+
|  aAcc | mIoU |  mAcc |
+-------+------+-------+
| 92.65 | 86.8 | 91.93 |
+-------+------+-------+
